In [6]:
import contextlib
import os
import string
from random import choices

import pandas as pd
from constants import max_steps, replicates

from mesa.batchrunner import batch_run_not_stupid
from mesa.examples.advanced.epstein_civil_violence.model import EpsteinCivilViolence

# Measure the sensitivity
This notebook is used to generate the data needed for the global sensitivity analysis. 

Be sure to first set the correct parameters in `constants.py` and to generate the parameter samples with `generate_parameters.ipynb`.

The parameter samples will be loaded from the folder given in `input_dir` and measurements will be saved to a folder given in `outputs_new`. Output files will be prefixed by an unique identifier to distinguish different runs.

The last cell in this notebook can be ran until ~~you are bored~~ enough data has been collected.

In [ ]:
# the folder containing the parameter samples
input_dir: str = "inputs"
# the folder to save the measurement results to
output_dir: str = "outputs"

# must be set to include all of the desired input files
# to resume a previous run, set `start` to nonzero value
start: int = 0
end: int = 20

files = [f"data_{x:04}.csv" for x in range(start, end)]
files

['data_0000.csv',
 'data_0001.csv',
 'data_0002.csv',
 'data_0003.csv',
 'data_0004.csv',
 'data_0005.csv',
 'data_0006.csv',
 'data_0007.csv',
 'data_0008.csv',
 'data_0009.csv',
 'data_0010.csv',
 'data_0011.csv',
 'data_0012.csv',
 'data_0013.csv',
 'data_0014.csv',
 'data_0015.csv',
 'data_0016.csv',
 'data_0017.csv',
 'data_0018.csv',
 'data_0019.csv']

In [8]:
def in_file(filename: str):
    """Generates the path to the given input file.
    Args:
        filename: the name of the file"""
    return f"{input_dir}/{filename}"


def out_file(filename: str, identifier: str):
    """Generates the path to the given output file.
    Args:
        filename: the name of the file.
        identifier: an unique identifier string add to the front of the name of
            the new file"""
    return f"{output_dir}/{identifier}_{filename}"

In [9]:
def run_for_file(filename: str, identifier: str):
    """
    Calculate and save the results of simulations with the parameters in the file.
    Args:
        filename: the name of the parameter file
        identifier: an unique identifier to use when saving the results
    """
    data = pd.read_csv(in_file(filename), index_col=0)
    print(f"Loaded {filename} with {len(data)} parameter samples.")

    result = batch_run_not_stupid(
        EpsteinCivilViolence,
        parameters=data,
        iterations=replicates,
        max_steps=max_steps,
        number_processes=None,
        display_progress=True,
    )

    result_df = pd.DataFrame(result)

    # we just want the folder to exist
    with contextlib.suppress(FileExistsError):
        os.mkdir(output_dir)
    result_df.to_csv(out_file(filename, identifier))

In [ ]:
while True:
    identifier = "".join(choices(string.ascii_letters + string.digits, k=8))
    print(f"Starting run {identifier}.")
    for file in files:
        run_for_file(file, identifier)